### KIVA LOANS Project

In [36]:
#Import Required libraries

import pandas as pd
import numpy as np
import datetime
import os

In [3]:
#pip install pypyodbc

In [37]:
data = pd.read_csv(r'kiva_loans.csv')
data.head(1)

,id,funded_amount,loan_amount,activity,sector,purpose,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,entryDate
0,653051,300,300,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12,12,NaN,female,irregular,2014-01-01


In [38]:
data.dtypes

id                      int64
funded_amount           int64
loan_amount             int64
activity               object
sector                 object
purpose                object
country_code           object
country                object
region                 object
currency               object
partner_id            float64
posted_time            object
disbursed_time         object
funded_time            object
term_in_months          int64
lender_count            int64
tags                   object
borrower_genders       object
repayment_interval     object
entryDate              object
dtype: object

In [6]:
data['disbursed_time'] = pd.to_datetime(data['disbursed_time'])
data['funded_time'] = pd.to_datetime(data['funded_time'])
data['posted_time'] = pd.to_datetime(data['posted_time'])

data.head(1)



,id,funded_amount,loan_amount,activity,sector,purpose,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,entryDate
0,653051,300,300,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12,12,NaN,female,irregular,2014-01-01


In [7]:
#data.dtypes

In [8]:
# Extract date and time components into new columns
data['disbursed_date'] = data['disbursed_time'].dt.date
data['disbursed_timeofday'] = data['disbursed_time'].dt.time
data['funded_date'] = data['funded_time'].dt.date
data['funded_time_timeofday'] = data['funded_time'].dt.time
#data['posted_date'] = data['posted_time'].dt.date
#data['posted_time_timeofday'] = data['posted_time'].dt.time
data.head(1)

,id,funded_amount,loan_amount,activity,sector,purpose,country_code,country,region,currency,...,term_in_months,lender_count,tags,borrower_genders,repayment_interval,entryDate,disbursed_date,disbursed_timeofday,funded_date,funded_time_timeofday
0,653051,300,300,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,...,12,12,NaN,female,irregular,2014-01-01,2013-12-17,08:00:00,2014-01-02,10:06:32


In [13]:
data.head(2)

,id,funded_amount,loan_amount,activity,sector,purpose,country_code,country,region,currency,...,term_in_months,lender_count,tags,borrower_genders,repayment_interval,entryDate,disbursed_date,disbursed_timeofday,funded_date,funded_time_timeofday
0,653051,300,300,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,...,12,12,NaN,female,irregular,2014-01-01,2013-12-17,08:00:00,2014-01-02,10:06:32
1,653053,575,575,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,...,11,14,NaN,"female, female",irregular,2014-01-01,2013-12-17,08:00:00,2014-01-02,09:17:23


In [17]:
from sqlalchemy import create_engine


Trying PYpodbc

In [24]:
import pyodbc

#Loop through all drivers we have access to
for driver in pyodbc.drivers():
    print(driver)


SQL Server
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access Text Driver (*.txt, *.csv)
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server


#### Test code to verify connection to SQL server

In [27]:
import pyodbc
import textwrap

# Define the Price Data.
price_data = [
    [ 2.00, 3.00, 1.00, 2.40, 100.00, '1/2/2019'],
    [ 3.00, 3.00, 5.00, 9.40, 300.00, '2/1/2020'],
    [ 4.00, 2.00, 1.00, 2.40, 200.00, '3/1/2021']
]

# define the server and the database
server = 'localhost' 
database = 'KivaLoans'  

# Define the connection string
cnxn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server}; \
    SERVER='+ server +'; \
    DATABASE='+ database +';\
    Trusted_Connection=yes;'
)

# Create the Cursor.
cursor = cnxn.cursor()

# Loop through to insert each row.
for index, row in enumerate(price_data):
    
    # define an insert query with place holders for the values.
    insert_query = textwrap.dedent('''
        INSERT INTO td_price_data (close_price, high, low, open_price, volume, day_value) 
        VALUES (?, ?, ?, ?, ?, ?);
    ''')
    
    # define the values
    values = (row[0], row[1], row[2], row[3], row[4], row[5])
    
    # insert the data into the database
    cursor.execute(insert_query, values)

# commit the inserts.
cnxn.commit()
    
# grab all the rows from the table
cursor.execute('SELECT * FROM td_price_data')
for row in cursor:
    print(row)
    
# close the cursor and connection  
cursor.close()
cnxn.close()

(2.0, 3.0, 1.0, 2.4, 100.0, datetime.date(2019, 1, 2))
(3.0, 3.0, 5.0, 9.4, 300.0, datetime.date(2020, 2, 1))
(4.0, 2.0, 1.0, 2.4, 200.0, datetime.date(2021, 3, 1))


##### Connecting to database to write data to sql

In [32]:
import pyodbc

# Define the connection string
 # define the server and the database
server = 'localhost' 
database = 'KivaLoans'  

# Define the connection string
cnxn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server}; \
    SERVER='+ server +'; \
    DATABASE='+ database +';\
    Trusted_Connection=yes;'
)

# Create the Cursor.
#cursor = cnxn.cursor()
# Establish the connection
try:
    connection = cnxn
    print("Connection successful!")
except pyodbc.Error as ex:
    print("Connection failed: ", ex)


Connection successful!


In [19]:
#pip install bcpandas

In [39]:
 #Create the connection string for SQLAlchemy
from sqlalchemy import create_engine

#Define server and database name
server = 'localhost' 
database = 'KivaLoans' 

#connection string
connection_string = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Write the DataFrame to a new SQL table
table_name = 'kiva_loans_main'
try:
    # 'if_exists' parameter can be 'fail', 'replace', or 'append'
    data.to_sql(name='kiva_loans_main', con=engine, if_exists='replace', index=False)
    print(f"Table '{table_name}' created successfully in the '{database}' database.")
except Exception as e:
    print(f"An error occurred: {e}")
    
    

Table 'kloans' created successfully in the 'KivaLoans' database.


In [40]:
# Writing data into database
import pandas as pd
from bcpandas import to_sql
from sqlalchemy import create_engine



#Define server and database name
server = 'localhost' 
database = 'KivaLoans' 

# Create the connection string for SQLAlchemy
connection_string = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Define the table name
table_name = 'kiva_loans_main'



# Use bcpandas to write the DataFrame to the SQL table
try:
    to_sql(df=data, name=table_name, schema='dbo', con=engine, if_exists='replace', index=False)
    print(f"Table '{table_name}' created successfully in the '{database}' database.")
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: to_sql() got an unexpected keyword argument 'name'


In [1]:
list_with_duplicates = [1,1,1,1,2,2,2,3,3,3,3,4,4,5]# Define a list with some duplicate values

# Use a set to find unique values
unique_values = set(list_with_duplicates)

# Print unique values
print("Unique values in the list are:", unique_values)

Unique values in the list are: {1, 2, 3, 4, 5}


#### Trying extracting data from an API source

In [1]:
import requests
import pandas as pd

url = 'https://search.worldbank.org/api/v2/wds?format=json&qterm=wind%20turbine&fl=docdt,count'
r = requests.get(url)
df = pd.DataFrame(r.json())
df.head()


,rows,os,page,total,documents
D11831032,10,0,1,546,"{'id': '11831032', 'count': 'Mexico', 'entityi..."
D15615154,10,0,1,546,"{'id': '15615154', 'count': 'Latin America', '..."
D19030958,10,0,1,546,"{'id': '19030958', 'count': 'Yemen, Republic o..."
D19544300,10,0,1,546,"{'id': '19544300', 'count': 'World', 'entityid..."
D26951778,10,0,1,546,"{'id': '26951778', 'count': 'World', 'entityid..."


In [2]:
df1 = df['documents'].explode()
df1.head()

D11831032           id
D11831032        count
D11831032    entityids
D11831032        docdt
D11831032    abstracts
Name: documents, dtype: object

In [3]:
data = r.json()
entries = data["documents"]
df2 = pd.DataFrame(entries)
df2

,D11831032,D29900830,D15615154,D19544300,D31211834,D19030958,D31211780,D26951778,D5704111,D698043,facets
id,11831032,29900830,15615154,19544300,31211834,19030958,31211780,26951778,5704111,698043,NaN
count,Mexico,Zambia,Latin America,World,Maldives,"Yemen, Republic of",Maldives,World,East Asia and Pacific,Costa Rica,NaN
entityids,{'entityid': '000334955_20100222012800'},{'entityid': '090224b08608649a_3_0'},{'entityid': '000386194_20120109010935'},{'entityid': '000333037_20140521124748'},{'entityid': '090224b086e466ac_1_0'},{'entityid': '000442464_20140220094225'},{'entityid': '090224b086e466ae_1_0'},{'entityid': '090224b084710367_1_0'},{'entityid': '000090341_20050401090154'},{'entityid': '000009265_3961219144813'},NaN
docdt,2006-04-24T00:00:00Z,2018-05-01T00:00:00Z,2011-12-01T00:00:00Z,2014-05-21T00:00:00Z,2018-10-01T00:00:00Z,2014-02-11T00:00:00Z,2019-06-28T00:00:00Z,2015-08-07T00:00:00Z,2000-01-01T00:00:00Z,1994-09-30T00:00:00Z,NaN
abstracts,{'cdata!': 'The objective of the Wind Umbrella...,{'cdata!': 'The overall Zambia ESMAP program ...,{'cdata!': 'This report identifies good practi...,{'cdata!': 'The kinetic energy in wind is conv...,{'cdata!': 'This report presents a description...,{'cdata!': 'The development objective of the M...,{'cdata!': 'The World Bank (the ‘Customer’) re...,"{'cdata!': 'The environmental, health, and saf...",{'cdata!': 'This paper presents the results of...,{'cdata!': 'The overall objectives of the Tejo...,NaN
display_title,Mexico - Wind Umbrella (La Venta\n ...,Renewable energy wind mapping for\n ...,Greening the wind : environmental\n ...,Implementing onshore wind power projects,Wind Resource Mapping in Maldives\n ...,Yemen - Mocha Wind Park Project,Wind Resource Mapping in Maldives\n ...,"Environmental, health, and safety\n ...",Statistical analysis of wind farm\n ...,Costa Rica - Tejona Wind Power Project,NaN
pdfurl,http://documents.worldbank.org/curated/en/9953...,http://documents.worldbank.org/curated/en/5287...,http://documents.worldbank.org/curated/en/2398...,http://documents.worldbank.org/curated/en/5565...,http://documents.worldbank.org/curated/en/2711...,http://documents.worldbank.org/curated/en/1567...,http://documents.worldbank.org/curated/en/6690...,http://documents.worldbank.org/curated/en/4988...,NaN,NaN,NaN
listing_relative_url,/projects/documents/2006/04/11831032/mexico-wi...,/research/2018/05/29900830/renewable-energy-wi...,/research/2011/12/15615154/greening-wind-envir...,/research/2014/05/19544300/implementing-onshor...,/research/2018/10/31211834/wind-resource-mappi...,/projects/documents/2014/02/19030958/yemen-moc...,/research/2019/06/31211780/wind-resource-mappi...,/research/2015/08/26951778/environmental-healt...,/research/2000/01/5704111/statistical-analysis...,/projects/documents/1994/09/698043/costa-rica-...,NaN
url_friendly_title,http://documents.worldbank.org/curated/en/9953...,http://documents.worldbank.org/curated/en/5287...,http://documents.worldbank.org/curated/en/2398...,http://documents.worldbank.org/curated/en/5565...,http://documents.worldbank.org/curated/en/2711...,http://documents.worldbank.org/curated/en/1567...,http://documents.worldbank.org/curated/en/6690...,http://documents.worldbank.org/curated/en/4988...,http://documents.worldbank.org/curated/en/4601...,http://documents.worldbank.org/curated/en/3462...,NaN
new_url,2006/04/11831032/mexico-wind-umbrella-la-venta...,2018/05/29900830/renewable-energy-wind-mapping...,2012/01/15615154/greening-wind-environmental-s...,2014/03/19544300/implementing-onshore-wind-pow...,2018/10/31211834/wind-resource-mapping-maldive...,2014/02/19030958/yemen-mocha-wind-park-project,2019/06/31211780/wind-resource-mapping-maldive...,2017/05/26951778/environmental-health-safety-g...,2000/01/5704111/statistical-analysis-wind-farm...,1994/09/698043/costa-rica-tejona-wind-power-pr...,NaN


In [4]:
df2.T

,id,count,entityids,docdt,abstracts,display_title,pdfurl,listing_relative_url,url_friendly_title,new_url,guid,url
D11831032,11831032,Mexico,{'entityid': '000334955_20100222012800'},2006-04-24T00:00:00Z,{'cdata!': 'The objective of the Wind Umbrella...,Mexico - Wind Umbrella (La Venta\n ...,http://documents.worldbank.org/curated/en/9953...,/projects/documents/2006/04/11831032/mexico-wi...,http://documents.worldbank.org/curated/en/9953...,2006/04/11831032/mexico-wind-umbrella-la-venta...,995361468056343658,http://documents.worldbank.org/curated/en/9953...
D29900830,29900830,Zambia,{'entityid': '090224b08608649a_3_0'},2018-05-01T00:00:00Z,{'cdata!': 'The overall Zambia ESMAP program ...,Renewable energy wind mapping for\n ...,http://documents.worldbank.org/curated/en/5287...,/research/2018/05/29900830/renewable-energy-wi...,http://documents.worldbank.org/curated/en/5287...,2018/05/29900830/renewable-energy-wind-mapping...,528711526549758961,http://documents.worldbank.org/curated/en/5287...
D15615154,15615154,Latin America,{'entityid': '000386194_20120109010935'},2011-12-01T00:00:00Z,{'cdata!': 'This report identifies good practi...,Greening the wind : environmental\n ...,http://documents.worldbank.org/curated/en/2398...,/research/2011/12/15615154/greening-wind-envir...,http://documents.worldbank.org/curated/en/2398...,2012/01/15615154/greening-wind-environmental-s...,239851468089382658,http://documents.worldbank.org/curated/en/2398...
D19544300,19544300,World,{'entityid': '000333037_20140521124748'},2014-05-21T00:00:00Z,{'cdata!': 'The kinetic energy in wind is conv...,Implementing onshore wind power projects,http://documents.worldbank.org/curated/en/5565...,/research/2014/05/19544300/implementing-onshor...,http://documents.worldbank.org/curated/en/5565...,2014/03/19544300/implementing-onshore-wind-pow...,556561468337209595,http://documents.worldbank.org/curated/en/5565...
D31211834,31211834,Maldives,{'entityid': '090224b086e466ac_1_0'},2018-10-01T00:00:00Z,{'cdata!': 'This report presents a description...,Wind Resource Mapping in Maldives\n ...,http://documents.worldbank.org/curated/en/2711...,/research/2018/10/31211834/wind-resource-mappi...,http://documents.worldbank.org/curated/en/2711...,2018/10/31211834/wind-resource-mapping-maldive...,271161564037906304,http://documents.worldbank.org/curated/en/2711...
D19030958,19030958,"Yemen, Republic of",{'entityid': '000442464_20140220094225'},2014-02-11T00:00:00Z,{'cdata!': 'The development objective of the M...,Yemen - Mocha Wind Park Project,http://documents.worldbank.org/curated/en/1567...,/projects/documents/2014/02/19030958/yemen-moc...,http://documents.worldbank.org/curated/en/1567...,2014/02/19030958/yemen-mocha-wind-park-project,156751468335513164,http://documents.worldbank.org/curated/en/1567...
D31211780,31211780,Maldives,{'entityid': '090224b086e466ae_1_0'},2019-06-28T00:00:00Z,{'cdata!': 'The World Bank (the ‘Customer’) re...,Wind Resource Mapping in Maldives\n ...,http://documents.worldbank.org/curated/en/6690...,/research/2019/06/31211780/wind-resource-mappi...,http://documents.worldbank.org/curated/en/6690...,2019/06/31211780/wind-resource-mapping-maldive...,669001564037551160,http://documents.worldbank.org/curated/en/6690...
D26951778,26951778,World,{'entityid': '090224b084710367_1_0'},2015-08-07T00:00:00Z,"{'cdata!': 'The environmental, health, and saf...","Environmental, health, and safety\n ...",http://documents.worldbank.org/curated/en/4988...,/research/2015/08/26951778/environmental-healt...,http://documents.worldbank.org/curated/en/4988...,2017/05/26951778/environmental-health-safety-g...,498831479463882556,http://documents.worldbank.org/curated/en/4988...
D5704111,5704111,East Asia and Pacific,{'entityid': '000090341_20050401090154'},2000-01-01T00:00:00Z,{'cdata!': 'This paper presents the results of...,Statistical analysis of wind farm\n ...,NaN,/research/2000/01/5704111/statistical-analysis...,http://documents.worldbank.org/curated/en/4601...,2000/01/5704111/

In [9]:


# Reset the index to bring the index as a column
#new_df.reset_index(inplace=True)

#MElt the DataFrame
#df_melted = pd.melt(df2, id_vars=['id'], value_vars=['count', 'entityids', 'docdt', 'abstracts', 'display_title', 'pdfurl', 'listing_relative_url','url_friendly_title',
                             # 'new_url', 'guid', 'url'],
                    #var_name='variable', value_name='value')

# Display the resulting DataFrame
#print(df_melted.head())
